# LIBRARIES

In [1]:
from IPython.display import display, Markdown
from bs4 import BeautifulSoup
import requests
from datetime import date
from datetime import datetime
from dotenv import load_dotenv
import locale
import os
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
os.environ["TOKENIZERS_PARALLELISM"] = "false" # to supress specific warning


from llama_index.core import VectorStoreIndex, get_response_synthesizer, Settings
from llama_index.core.schema import TextNode
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

locale.setlocale(locale.LC_ALL, "en_US")
load_dotenv()

c:\Users\Dell\miniconda3\envs\AIbuilders\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
Settings.llm = OpenAI(model = "gpt-4o-mini",
                      api_key = os.getenv('OPEN_API_KEY'),
                      temperature = 0.5)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [3]:
limit = datetime.strptime("January 01, 2020", "%B %d, %Y")  
date_wp = datetime.strptime("May 02, 2025", "%B %d, %Y")  
page_number = 1
titles = []
abstracts = []
dates = []

while date_wp >= limit:
    url = requests.get(f'https://www.imf.org/en/Publications/WP/?page={page_number}').text
    page = BeautifulSoup(url, 'html.parser')

    w_papers = page.find_all("div", {'class':'result-row pub-row'})

    for paper in w_papers:
        date_paper = paper.find_all("p")[0].text.strip()    
        
        if date_paper != 'January 1, 0001':
            title = paper.find_all("a")[0].text.strip()
            abstract = paper.find_all("span")[0].text.strip()            
            titles.append(title)
            abstracts.append(abstract)
            dates.append(date_paper)
        else:
            date_paper = "May 02, 2025"
    
    date_wp = datetime.strptime(date_paper, "%B %d, %Y")
    page_number += 1

In [4]:
articles_nodes = []

for i in range(len(titles)):
    articles_nodes.append(
        TextNode(
            id = str(i),
            text = abstracts[i],
            metadata = {
                "title" : titles[i],
                "date" : dates[i]
            }
        )
    )

In [5]:
index = VectorStoreIndex(articles_nodes, show_progress = True)

Generating embeddings: 100%|██████████| 1479/1479 [04:43<00:00,  5.21it/s]


In [6]:
print(f"Embedding Model: {index._embed_model.model_name}")
print(f"Index Size: {len(index.vector_store.data.embedding_dict)}")
#print(f"Embedding Size: {len(index.vector_store.data.embedding_dict["0"])}")

Embedding Model: BAAI/bge-small-en-v1.5
Index Size: 1479


In [7]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k = 10,
)

In [8]:
results = retriever.retrieve("What could be the possible effects of AI in society?")

In [16]:
# format results in markdown
results_markdown = ""
for i, result in enumerate(results, start=1):
    results_markdown += f"{i}. **Article title:** {result.metadata["title"]}  \n"
    results_markdown += f"   **Date:** {result.metadata["date"]}  \n"
    results_markdown += f"   **Abstract:** {result.text}  \n\n"

In [17]:
display(Markdown(results_markdown))

1. **Article title:** The Economic Impacts and the Regulation of AI: A Review of the Academic Literature and Policy Actions  
   **Date:** March 22, 2024  
   **Abstract:** We review the literature on the effects of Artificial Intelligence (AI) adoption and the ongoing regulatory efforts concerning this technology. Economic research encompasses growth, employment, productivity, and income inequality effects, while regulation covers market competition, data privacy, copyright, national security, ethics concerns, and financial stability. We find that: (i) theoretical research agrees that AI will affect most occupations and transform growth, but empirical findings are inconclusive on employment and productivity effects; (ii) regulation has focused primarily on topics not explored by the academic literature; (iii) across countries, regulations differ widely in scope and approaches and face difficult trade-offs.  

2. **Article title:** The Labor Market Impact of Artificial Intelligence: Evidence from US Regions  
   **Date:** September 13, 2024  
   **Abstract:** This paper empirically investigates the impact of Artificial Intelligence (AI) on employment. Exploiting variation in AI adoption across US commuting zones using a shift-share approach, I find that during 2010-2021, commuting zones with higher AI adoption have experienced a stronger decline in the employment-to-population ratio. Moreover, this negative employment effect is primarily borne by the manufacturing and lowskill services sectors, middle-skill workers, non-STEM occupations, and individuals at the two ends of the age distribution. The adverse impact is also more pronounced on men than women.  

3. **Article title:** AI Adoption and Inequality  
   **Date:** April 4, 2025  
   **Abstract:** There are competing narratives about artificial intelligence’s impact on inequality. Some argue AI will exacerbate economic disparities, while others suggest it could reduce inequality by primarily disrupting high-income jobs. Using household microdata and a calibrated task-based model, we show these narratives reflect different channels through which AI affects the economy. Unlike previous waves of automation that increased both wage and wealth inequality, AI could reduce wage inequality through the displacement of high-income workers. However, two factors may counter this effect: these workers’ tasks appear highly complementary with AI, potentially increasing their productivity, and they are better positioned to benefit from higher capital returns. When firms can choose how much AI to adopt, the wealth inequality effect is particularly pronounced, as the potential cost savings from automating high-wage tasks drive significantly higher adoption rates. Models that ignore this adoption decision risk understating the trade-off policymakers face between inequality and efficiency.  

4. **Article title:** The Global Impact of AI: Mind the Gap  
   **Date:** April 11, 2025  
   **Abstract:** This paper examines the uneven global impact of AI, highlighting how its effects will be a function of (i) countries’ sectoral exposure to AI, (ii) their preparedness to integrate these technologies into their economies, and (iii) their access to essential data and technologies. We feed these three aspects into a multi-sector dynamic general equilibrium model of the global economy and show that AI will exacerbate cross-country income inequality, disproportionately benefiting advanced economies. Indeed, the estimated growth impact in advanced economies could be more than double that in low-income countries. While improvements in AI preparedness and access can mitigate these disparities, they are unlikely to fully offset them. Moreover, the AI-driven productivity gains could reduce the traditional role of exchange rate adjustments due to AI’s large impact in the non-tradable sector—a mechanism akin to an inverse Balassa-Samuelson effect.  

5. **Article title:** Labor Market Exposure to AI: Cross-country Differences and Distributional Implications  
   **Date:** October 4, 2023  
   **Abstract:** This paper examines the impact of Artificial Intelligence (AI) on labor markets in both Advanced Economies (AEs) and Emerging Markets (EMs). We propose an extension to a standard measure of AI exposure, accounting for AI's potential as either a complement or a substitute for labor, where complementarity reflects lower risks of job displacement. We analyze worker-level microdata from 2 AEs (US and UK) and 4 EMs (Brazil, Colombia, India, and South Africa), revealing substantial variations in unadjusted AI exposure across countries. AEs face higher exposure than EMs due to a higher employment share in professional and managerial occupations. However, when accounting for potential complementarity, differences in exposure across countries are more muted. Within countries, common patterns emerge in AEs and EMs. Women and highly educated workers face greater occupational exposure to AI, at both high and low complementarity. Workers in the upper tail of the earnings distribution are more likely to be in occupations with high exposure but also high potential complementarity.  

6. **Article title:** Exposure to Artificial Intelligence and Occupational Mobility: A Cross-Country Analysis  
   **Date:** June 7, 2024  
   **Abstract:** We document historical patterns of workers' transitions across occupations and over the life-cycle for different levels of exposure and complementarity to Artificial Intelligence (AI) in Brazil and the UK. In both countries, college-educated workers frequently move from high-exposure, low-complementarity occupations (those more likely to be negatively affected by AI) to high-exposure, high-complementarity ones (those more likely to be positively affected by AI). This transition is especially common for young college-educated workers and is associated with an increase in average salaries. Young highly educated workers thus represent the demographic group for which AI-driven structural change could most expand opportunities for career progression but also highly disrupt entry into the labor market by removing stepping-stone jobs. These patterns of “upward” labor market transitions for college-educated workers look broadly alike in the UK and Brazil, suggesting that the impact of AI adoption on the highly educated labor force could be similar across advanced economies and emerging markets. Meanwhile, non-college workers in Brazil face markedly higher chances of moving from better-paid high-exposure and low-complementarity occupations to low-exposure ones, suggesting a higher risk of income loss if AI were to reduce labor demand for the former type of jobs.  

7. **Article title:** Will the AI Revolution Cause a Great Divergence?  
   **Date:** September 11, 2020  
   **Abstract:** This paper considers the implications for developing countries of a new wave of technological change that substitutes pervasively for labor. It makes simple and plausible assumptions: the AI revolution can be modeled as an increase in productivity of a distinct type of capital that substitutes closely with labor; and the only fundamental difference between the advanced and developing country is the level of TFP. This set-up is minimalist, but the resulting conclusions are powerful: improvements in the productivity of “robots” drive divergence, as advanced countries differentially benefit from their initially higher robot intensity, driven by their endogenously higher wages and stock of complementary traditional capital. In addition,  capital—if internationally mobile—is pulled “uphill”, resulting in a transitional GDP decline in the developing country. In an extended model where robots substitute only for unskilled labor, the terms of trade, and hence GDP, may decline permanently for the country relatively well-endowed in unskilled labor.  

8. **Article title:** Technological Progress, Artificial Intelligence, and Inclusive Growth  
   **Date:** June 11, 2021  
   **Abstract:** Advances in artificial intelligence and automation have the potential to be labor-saving and to increase inequality and poverty around the globe. They also give rise to winner-takes-all dynamics that advantage highly skilled individuals and countries that are at the forefront of technological progress. We analyze the economic forces behind these developments and delineate domestic economic policies to mitigate the adverse effects while leveraging the potential gains from technological advances. We also propose reforms to the global system of governance that make the benefits of advances in artificial intelligence more inclusive.  

9. **Article title:** AI and Productivity in Europe  
   **Date:** April 4, 2025  
   **Abstract:** The discussion on Artificial Intelligence (AI) often centers around its impact on productivity, but macroeconomic evidence for Europe remains scarce. Using the Acemoglu (2024) approach we simulate the medium-term impact of AI adoption on total factor productivity for 31 European countries. We compile many scenarios by pooling evidence on which tasks will be automatable in the near term, using reduced-form regressions to predict AI adoption across Europe, and considering relevant regulation that restricts AI use heterogeneously across tasks, occupations and sectors. We find that the medium-term productivity gains for Europe as a whole are likely to be modest, at around 1 percent cumulatively over five years. While economcially still moderate, these gains are still larger than estimates by Acemoglu (2024) for the US. They vary widely across scenarios and countries and are sustantially larger in countries with higher incomes. Furthermore, we show that national and EU regulations around occupation-level requirements, AI safety, and data privacy combined could reduce Europe’s productivity gains by over 30 percent if AI exposure were 50 percent lower in tasks, occupations and sectors affected by regulation.  

10. **Article title:** Strengthening Social Protection to Pave the Way for Technological Innovation: Evidence from the U.S.  
   **Date:** May 3, 2024  
   **Abstract:** This paper investigates the impact of automation on the U.S. labor market from 2000 to 2007, specifically examining whether more generous social protection programs can mitigate negative effects. Following Acemoglu and Restrepo (2020), the study finds that areas with higher robot adoption reduced employment and wages, in particular for workers without collegue degree. Notably, the paper exploits differences in social protection generosity across states and finds that areas with more generous unemployment insurance (UI) alleviated the negative effects on wages, especially for less-skilled workers. The results suggest that UI allowed displaced workers to find better matches The findings emphasize the importance of robust social protection policies in addressing the challenges posed by automation, contributing valuable insights for policymakers.  



In [18]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

response = query_engine.query("What could be the possible effects of AI in society?")
print(response)

The adoption of AI in society could lead to several significant effects. It is expected to transform various occupations and influence economic growth. However, the impact on employment and productivity remains uncertain, with some sectors experiencing declines in job opportunities, particularly in manufacturing and low-skill services. 

AI may also affect income inequality, with differing narratives suggesting it could either exacerbate or reduce disparities. While high-income jobs may face disruption, the potential for AI to enhance productivity among these workers could counterbalance wage inequality. Moreover, the global impact of AI is likely to be uneven, benefiting advanced economies more than low-income countries and exacerbating cross-country income inequality.

In terms of labor markets, exposure to AI varies across countries, with advanced economies facing higher risks due to their employment structures. Workers with higher education levels might experience more opportunitie

In [19]:
response = query_engine.query("What actions could a country implement to mititgate the negative effects of AI adoption?")
print(response)

To mitigate the negative effects of AI adoption, a country could implement several actions, including:

1. **Enhancing Education and Training**: Investing in education and vocational training programs to equip the workforce with the necessary skills to adapt to AI technologies. This includes reskilling and upskilling initiatives focused on high-demand areas.

2. **Promoting Inclusive Policies**: Developing policies that ensure equitable access to AI technologies and their benefits, particularly for low-income and marginalized communities. This could involve subsidies or support for small businesses to adopt AI.

3. **Implementing Safety Nets**: Establishing social safety nets such as universal basic income or unemployment benefits to support individuals displaced by AI automation, ensuring they have financial security during transitions.

4. **Encouraging Research and Development**: Supporting R&D in AI that focuses on creating technologies that complement rather than replace human lab